In [ ]:
######################################################################
###############   IMPORTING THE NECESSARY PACKAGES  ##################
######################################################################

import torch
import torch.nn as nn
import numpy as np
import os

# clip_grad_norm_ is used for truncated backpropagation
from torch.nn.utils import clip_grad_norm_


In [ ]:
######################################################################
##################   DEVICE CONFIGURATION ############################
######################################################################

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
######################################################################
############ DEFINING THE DICTIIONARY AND CORPUS CLASSES #############
######################################################################

class Dictionary(object):
    
    def __init__(self):
        # Two dictionaries maintained
        # For finding the index of a given word - word2idx
        self.word2idx = {}
        # For finding the word given an index - idx2word
        self.idx2word = {}
        self.idx = 0
    
    # Method for adding a new word into an instance of the Dictionary class.
    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
            
    # method will return the vocabulary size.
    def __len__(self):
        return(len(self.word2idx))
    
class Corpus(object):
    def __init__(self):
        self.Dictionary = Dictionary()
        
    def get_data(self,path, batch_size=20):
        # Add word to the dictionary
        with open(path,'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words:
                    self.Dictionary.add_word(word)
        
        
        # Tokenize the file content
        ids = torch.LongTensor(tokens)
        token = 0
        with open(path, 'r') as f:
            for line in f:
                words = line.split() + ['<eos>']
                for words in words:
                    ids[token] = self.Dictionary.word2idx[word]
                    token += 1 
        num_batches = ids.size(0) // batch_size
        ids = ids[:num_batches*(batch_size)]
        return(ids.view(batch_size, -1))
    

In [ ]:
######################################################################
##################  HYPTERPARAMETER DEFINITION #######################
######################################################################

batch_size = 20
embed_size = 128
hidden_size = 1024
num_layers = 1
num_epochs = 2

num_samples = 1000
seq_length = 30
learning_rate = 0.002


In [ ]:
######################################################################
##############  LOADING THE PENN TREEBANK DATASET ####################
######################################################################

path = 'data/train.txt'
corpus = Corpus()
idxs = corpus.get_data(path, batch_size)
vocab_size = len(corpus.Dictionary)
num_batches = idxs.size(1) // seq_length

In [ ]:
######################################################################
##############  DEFINING RNN BASED LANGUAGE MODEL ####################
######################################################################

class RNNLM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(RNNLM, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embed_size)
        self.LSTM = nn.LSTM(embed_size, hidden_size, num_layers, batch_first = True)
        self.out = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, x, h):
        
        # Taking word indices to embeddings
        x = self.embeddings(x)
        
        # Forward propagate LSTM
        out, (h,c) = self.LSTM(x, h)
        
        # Reshaping the output in desired shape (batch_size*sequence_length, hidden_size)
        out = out.reshape(out.size(0)*out.size(1), out.size(2))
        
        # Decoding to the final vocabulary size to get the word
        out = self.out(out)
        
        return(out, (h,c))
    
Lang_Model = RNNLM(vocab_size, embed_size, hidden_size, num_layers).to(device)

In [ ]:
######################################################################
#######################  LOSS AND OPTIMIZER ##########################
######################################################################

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(Lang_Model.parameters(), lr = learning_rate)

In [ ]:
######################################################################
#################### TRUNCATED BACKPROPAGATION #######################
######################################################################

def detach(states):
    return [state.detach() for state in states]

In [ ]:
######################################################################
####################### TRAINING THE MODEL ###########################
######################################################################

for epoch in range(num_epochs):
    # Set initial hidden and cell states
    states = (torch.zeros(num_layers, batch_size, hidden_size).to(device), torch.zeros(num_layers, batch_size, hidden_size).to(device))
    
    for i in range(0, idxs.size(1) - seq_length, seq_length):
        # Get mini-batch inputs and targets
        inputs = idxs[:, i:i+seq_length].to(device)
        targets = idxs[:, (i+1):(i+1)+seq_length].to(device)
        
        # Forward Pass
        states = detach(states)
        outputs, states = Lang_Model(inputs, states)
        loss = criterion(outputs, targets.reshape(-1))
        
        # Backpropagation
        Lang_Model.zero_grad()
        loss.backward()
        clip_grad_norm_(Lang_Model.parameters(), 0.5)
        optimizer.step()
        
        step = (i+1) // seq_length
        if step % 100 == 0:
            print('Epoch: [{}/{}], Step: [{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'.format(epoch+1,num_epochs, step, num_batches, loss.item(), np.exp(loss.item())))
        